In [21]:
import pandas as pd

products = pd.read_csv(r'C:\Users\dell\Desktop\img\products.csv')
products

,product_id,product_name,supplier_id,category_id,quantity_per_unit,unit_price,units_in_stock,units_on_order,reorder_level,discontinued
0,1,Chai,8,1,10 boxes x 30 bags,18.000,39,0,10,1
1,2,Chang,1,1,24 - 12 oz bottles,19.000,17,40,25,1
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.000,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.000,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.350,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
76,41,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,10.615,85,0,10,0
77,45,Rogede sild,21,8,1k pkg.,10.450,5,70,15,0
78,46,Spegesild,21,8,4 - 450 g glasses,13.200,95,0,0,0
79,58,Escargots de Bourgogne,27,8,24 pieces,14.575,62,0,20,0


In [22]:
categories = pd.read_csv(r'C:\Users\dell\Desktop\img\categories.csv')
categories

,category_id,category_name,description,picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",binary data
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",binary data
2,3,Confections,"Desserts, candies, and sweet breads",binary data
3,4,Dairy Products,Cheeses,binary data
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",binary data
5,6,Meat/Poultry,Prepared meats,binary data
6,7,Produce,Dried fruit and bean curd,binary data
7,8,Seafood,Seaweed and fish,binary data


In [8]:
data = categories.merge(products)
data

,category_id,category_name,description,picture,product_id,product_name,supplier_id,quantity_per_unit,unit_price,units_in_stock,units_on_order,reorder_level,discontinued
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",binary data,1,Chai,8,10 boxes x 30 bags,18.000,39,0,10,1
1,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",binary data,2,Chang,1,24 - 12 oz bottles,19.000,17,40,25,1
2,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",binary data,24,Guaraná Fantástica,10,12 - 355 ml cans,4.500,20,0,0,1
3,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",binary data,34,Sasquatch Ale,16,24 - 12 oz bottles,14.000,111,0,15,0
4,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",binary data,35,Steeleye Stout,16,24 - 12 oz bottles,18.000,20,0,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,8,Seafood,Seaweed and fish,binary data,41,Jack's New England Clam Chowder,19,12 - 12 oz cans,10.615,85,0,10,0
77,8,Seafood,Seaweed and fish,binary data,45,Rogede sild,21,1k pkg.,10.450,5,70,15,0
78,8,Seafood,Seaweed and fish,binary data,46,Spegesild,21,4 - 450 g glasses,13.200,95,0,0,0
79,8,Seafood,Seaweed and fish,binary data,58,Escargots de Bourgogne,27,24 pieces,14.575,62,0,20,0


In [18]:
q = """SELECT category_name, SUM(unit_price * units_in_stock)
FROM products
INNER JOIN categories ON products.category_id = categories.category_id
WHERE discontinued <> 1
GROUP BY category_name
HAVING SUM(unit_price * units_in_stock) > 5000
ORDER BY SUM(unit_price * units_in_stock) DESC;"""

In [19]:
from pandasql import sqldf 
sqldf(q)

,category_name,SUM(unit_price * units_in_stock)
0,Seafood,14311.385
1,Condiments,13590.050
2,Beverages,11365.250
3,Dairy Products,11271.200
4,Confections,10392.200
5,Grains/Cereals,5230.500


In [20]:
merged_df = pd.merge(categories, products)
filtered_df = merged_df[merged_df.discontinued != 1]
category_group = filtered_df.groupby('category_name').apply(lambda x: (x.units_in_stock*x.unit_price).agg(['sum']))
category_group=category_group[category_group['sum'] > 5000]
category_group.sort_values(by='sum', ascending=False)

,sum
category_name,
Seafood,14311.385
Condiments,13590.050
Beverages,11365.250
Dairy Products,11271.200
Confections,10392.200
Grains/Cereals,5230.500


In [35]:
q = """
SELECT category_id, category_name, sum(unit_price)
from data
group by category_id
order by sum(unit_price) desc"""
from pandasql import sqldf 
sqldf(q)

,category_id,category_name,sum(unit_price)
0,1,Beverages,455.750
1,3,Confections,327.080
2,2,Condiments,326.250
3,6,Meat/Poultry,324.040
4,4,Dairy Products,287.300
5,8,Seafood,273.009
6,7,Produce,161.850
7,5,Grains/Cereals,141.750


In [38]:
merged_df = pd.merge(categories, products)
category_group = merged_df.groupby(['category_id', 
                                    'category_name']).aggregate({'unit_price': 'sum'}).sort_values('unit_price', 
                                                                                                   ascending=False)
category_group.reset_index(1)

,category_name,unit_price
category_id,,
1,Beverages,455.750
3,Confections,327.080
2,Condiments,326.250
6,Meat/Poultry,324.040
4,Dairy Products,287.300
8,Seafood,273.009
7,Produce,161.850
5,Grains/Cereals,141.750
